In [1]:
import datetime
from math import ceil
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from pathlib import Path

from summer.utils import ref_times_to_dti

from autumn.settings.constants import COVID_BASE_DATETIME
from autumn.core.runs import ManagedRun
from autumn.core.project import get_project

C:\Users\ehug0006\Miniconda3\envs\autumn310\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\ehug0006\Miniconda3\envs\autumn310\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\ehug0006\Miniconda3\envs\autumn310\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
C:\Users\ehug0006\Miniconda3\envs\autumn310\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
region = "northern_territory"

In [3]:
run_id = "sm_sir/northern_territory/1662380460/28ff0b7"

In [4]:
mr = ManagedRun(run_id)

In [5]:
pbi = mr.powerbi.get_db()
results = pbi.get_uncertainty()

In [6]:
project = get_project("sm_sir", region, reload=True)

AssertionError: Project northern_territory not registered as a project using model sm_sir.

In [ ]:
title_lookup = {
    "notifications": "daily notifications",
    "infection_deaths": "COVID-19-specific deaths",
    "hospital_admissions": "new daily hospital admissions",
    "icu_admissions": "new daily admissions to ICU",
    "proportion_seropositive": "proportion recovered from COVID-19",
    "incidence": "daily new infections",
    "prop_incidence_strain_delta": "proportion of cases due to Delta",
    "hospital_admissions": "daily hospital admissions",
    "hospital_occupancy": "total hospital beds",
    "icu_admissions": "daily ICU admissions",
    "icu_occupancy": "total ICU beds",
    "prop_ever_infected": "ever infected with Delta or Omicron",
    "cumulative_infection_deaths": "cumulative COVID-19 deaths",
    "cumulative_hospital_admissions": "cumulative hospital admissions",
    "cumulative_icu_admissions": "cumulative ICU admissions",    
}

In [ ]:
def outputs_with_uncertainty(outputs, scenarios, input_shape=None):
    
    # Set row and col position for outputs based on requested outputs
    # If you want can specify own shape/layout
    if input_shape:
        shape = input_shape
    # Otherwise define shape based on requested outputs
    else:
        if len(outputs) == 4:
            shape = [2, 2]
        elif len(outputs) == 3:
            shape = [1, 3]
        elif len(outputs) == 2:
            shape = [1, 2]
        elif len(outputs) == 1:
            shape = [1, 1]
    
    # Store number of rows and columns
    rows = shape[0]
    cols = shape[1]
    
    # Create all [row, col] positions
    positions = []
    for r in range(rows):
        for c in range(cols):
            positions.append([r+1, c+1])
    
    # Create/setup figure, note need to set subplot_titles here to outputs, and then update after figure is made
    fig = make_subplots(rows, cols, subplot_titles=outputs)
    
    # Define colours for scenarios as partial rgba strings to allow for specifying opacity with alpha
    colours = {0: 'rgba(26,150,65,',
               1: 'rgba(248,156,116,',
               2: 'rgba(204, 102, 119,',
               3: 'rgba(29, 105, 150,',
               4: 'rgba(221, 204, 119,'
              }
    # Loop through to obtain position-output pairs for each scenario and plots
    for pos, o in zip(positions, outputs):
        for s in scenarios:
            colour = colours[s]
            
            # Set transparency for median and credible intervals here
            median_alpha = 1.0
            fifty_cred_alpha = 0.4
            ninety_five_cred_alpha = 0.2
            
            # Set data to use, index and labels
            results_df = results[(o, s)]
            indices = results_df.index
            label = "baseline" if s == 0 else project.param_set.scenarios[s - 1]["description"]
            show_legend = True if pos[0] == 1 and pos[1] == 1 else False
            
            # add median trace
            fig.add_trace(go.Scatter(x=indices, y=results_df[0.500], 
                                     line=dict(color=(colour+str(median_alpha)+")")), 
                                     name=label,
                                     showlegend = show_legend
                                    ),
                         row = pos[0],
                          col= pos[1]
                         )
            
            #50% CI trace
            fig.add_traces([go.Scatter(x=results_df.index, y=results_df[0.250], 
                                       line=dict(color=colour+str(fifty_cred_alpha)+")"),
                                       fillcolor=colour+str(fifty_cred_alpha)+")", showlegend=False),
                            go.Scatter(x=results_df.index, y=results_df[0.750], 
                                       line=dict(color=colour+str(fifty_cred_alpha)+")"), 
                                       fillcolor=colour+str(fifty_cred_alpha)+")", 
                                       fill="tonexty", showlegend=False)
                           ],
                           rows = pos[0],
                          cols = pos[1]
                          )
            
            #95% CI trace
            fig.add_traces([go.Scatter(x=results_df.index, y=results_df[0.025], 
                                       line=dict(color=colour+str(ninety_five_cred_alpha)+")"), 
                                       fillcolor=colour+str(ninety_five_cred_alpha)+")", showlegend=False),
                            go.Scatter(x=results_df.index, y=results_df[0.975], 
                                       line=dict(color=colour+str(ninety_five_cred_alpha)+")"),
                                       fillcolor=colour+str(ninety_five_cred_alpha)+")", 
                                       fill="tonexty", showlegend=False)
                           ],
                           rows = pos[0],
                          cols = pos[1]
                          )
    # Subplot titles stored as annotations and so need to update annotation text for each subplot;
    # Currently set to the name of the corresponding output, use title_lookup to set title
    #code pinched from user vestland: 
    #https://stackoverflow.com/questions/63220009/how-do-i-set-each-plotly-subplot-title-during-graph-creation-loop
    fig.for_each_annotation(lambda a: a.update(text = title_lookup[a.text]))
    
    # place legend in upper left hand corner
    fig.update_layout(legend_x=-0.05, legend_y=1.3,
                      # Need to adjust height and width according to number of outputs
                     height=400, width=800)
    
    # Update image config to allow for higher resolution image using quick downlaod image button on output
    config = {
        'toImageButtonOptions': {
            'format': 'png',
            'filename': 'image_1',
            'height': 400,
            'width': 1000,
            'scale':4 #adjust scale to increase resolution
        }
    }


    fig.show(config=config)
    
   

In [ ]:
# Specify scenarios here
scenarios = [0, 4]

# Specify outputs here up to 4: function will determine shape based on number of outputs specified
outputs = ('notifications',) #"hospital_occupancy",'icu_admissions', "infection_deaths")

outputs_with_uncertainty(outputs, scenarios)